# Pre-processing

In [ ]:
# To know the type of each column
df.dtypes
# reminder : our projects can't use string values

In [ ]:
df.describe()
# what to look for :   Highest and lowest values to notice the ootliers
# If any odd values were found, determine all uniqe values and replace the odd ones
apgar_5.unique()
apgar_5 = apgar_5.replace([10.5],10)

In [ ]:
df.groupby([df.combo].mean)

In [2]:
# Better yet is using sweetviz or pandas_profiling
# pandas_profiling
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport


#data = pd.read_excel (r'neonatal/neonatal outcomes.xlsx')
df = pd.read_excel('neonatal/neonatal outcomes.xlsx', engine='openpyxl')


profile = ProfileReport(df, title="Pandas Profiling Report")

profile.to_file("output.html")

In [ ]:
# sweetviz
import sweetviz as sv
#analyzing the dataset
car_report = sv.analyze(df)
car_report.show_html('Report.html')
#Analyzing a single dataframe (and its -optional- target feature)
my_report = sv.analyze([train, "Train"],target_feat='SalePrice')
my_report.show_html('Report.html')
#Comparing two dataframes (e.g. Test vs Training sets)
my_report1 = sv.compare([train, "Train"], [test, "Test"], "SalePrice")
my_report1.show_html('Report.html')


In [ ]:
# showing the html files in python
import IPython
IPython.display.HTML('Report.html')

In [ ]:
#STEROID(S) One hot encoding
steroids_dum=pd.get_dummies(df.steroids)
steroids_dum.columns = ['steroids1', 'steroids2', 'steroids3']
df_final = pd.concat([df,steroids_dum],axis='columns')

In [ ]:
# Feature selection 
import pandas as pd
import numpy as np
import random as rd
from sklearn.decomposition import PCA
from sklearn import preprocessing
import matplotlib.pyplot as plt # NOTE: This was tested with matplotlib v. 2.1.0
 
#########################
#
# Data Generation Code
#
#########################
## In this example, the data is in a data frame called data.
## Columns are individual samples (i.e. cells)
## Rows are measurements taken for all the samples (i.e. genes)
## Just for the sake of the example, we'll use made up data...
genes = ['gene' + str(i) for i in range(1,101)]
 
wt = ['wt' + str(i) for i in range(1,6)]
ko = ['ko' + str(i) for i in range(1,6)]
 
data = pd.DataFrame(columns=[*wt, *ko], index=genes)
 
for gene in data.index:
    data.loc[gene,'wt1':'wt5'] = np.random.poisson(lam=rd.randrange(10,1000), size=5)
    data.loc[gene,'ko1':'ko5'] = np.random.poisson(lam=rd.randrange(10,1000), size=5)
 
print(data.head())
print(data.shape)
 
#########################
#
# Perform PCA on the data
#
#########################
# First center and scale the data
scaled_data = preprocessing.scale(data.T)
 
pca = PCA() # create a PCA object
pca.fit(scaled_data) # do the math
pca_data = pca.transform(scaled_data) # get PCA coordinates for scaled_data
 
#########################
#
# Draw a scree plot and a PCA plot
#
#########################
 
#The following code constructs the Scree plot
per_var = np.round(pca.explained_variance_ratio_* 100, decimals=1)
labels = ['PC' + str(x) for x in range(1, len(per_var)+1)]
 
plt.bar(x=range(1,len(per_var)+1), height=per_var, tick_label=labels)
plt.ylabel('Percentage of Explained Variance')
plt.xlabel('Principal Component')
plt.title('Scree Plot')
plt.show()
 
#the following code makes a fancy looking plot using PC1 and PC2
pca_df = pd.DataFrame(pca_data, index=[*wt, *ko], columns=labels)
 
plt.scatter(pca_df.PC1, pca_df.PC2)
plt.title('My PCA Graph')
plt.xlabel('PC1 - {0}%'.format(per_var[0]))
plt.ylabel('PC2 - {0}%'.format(per_var[1]))
 
for sample in pca_df.index:
    plt.annotate(sample, (pca_df.PC1.loc[sample], pca_df.PC2.loc[sample]))
 
plt.show()
 
#########################
#
# Determine which genes had the biggest influence on PC1
#
#########################
 
## get the name of the top 10 measurements (genes) that contribute
## most to pc1.
## first, get the loading scores
loading_scores = pd.Series(pca.components_[0], index=genes)
## now sort the loading scores based on their magnitude
sorted_loading_scores = loading_scores.abs().sort_values(ascending=False)
 
# get the names of the top 10 genes
top_10_genes = sorted_loading_scores[0:10].index.values
 
## print the gene names and their scores (and +/- sign)
print(loading_scores[top_10_genes])

In [ ]:
# remove outliers (best with continous data)
max_thresold = df['height'].quantile(0.95)
min_thresold = df['height'].quantile(0.05)
df = df[(df['height']<max_thresold) & (df['height']>min_thresold)]

In [ ]:
#word to num
from word2number import w2n
df.x = x2n.word_to_num(df.x)

In [ ]:
# Missing Values

# plot missing features
from feature_selector import FeatureSelector
fs = FeatureSelector(data = X, labels = train_labels)
fs.identify_missing(missing_threshold=0.6)
missing_features = fs.ops['missing']
missing_features[:10]
fs.plot_missing()


# Analyze how many null values are in the dataset
# import math
#data.isnull().sum()
# data.plot_missing()

# drop all missing values in all columns:
#data = data.dropna()

# Mean value imputation:
# data["Age"].fillna(data["Age"].mean(), inplace=True)

# Check the database after missed data were dropped
#data.shape

In [ ]:
#implanced data
ones = data[data['any_outcome']==1]
zeros = data[data['any_outcome']==0]
print(ones.shape,zeros.shape)

# لو طلع في فرق بين الرقمين

# Implementing Undersampling بياخد عينة عشوائية من المجموعة الكبيرة
nm = NearMiss(random_state=42)
X_res,y_res=nm.fit_sample(X,Y)

# Implementing Oversampling (بيضاعف المجموعة الصغيرة شبه البوتستراب)
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss
smk = SMOTETomek(random_state=42)
X_res,y_res=smk.fit_sample(X,Y)

In [ ]:
#Composite Outcome
C_outcome = df['neonatal_morbidity'] + df ['perinatal_death'] + df['nicu_admission'] + df['resp_support'] + df['apgar_5']
C_outcome = C_outcome.replace([2,3,4,5],1)
C_outcome.columns = ['C_outcome']

In [ ]:
# some algorithms work better with normalization (better with continous data)
Features =  np.asarray(df_final[['age', 'parity', 'bmi']])


In [ ]:
# Name every outcome differently